## Import libraries

In [81]:
import datetime
import pandas as pd
from igf_data.igfdb.igfTables import Base, Project, User, Platform, Sample, Experiment, Seqrun, Run, Seqrun_stats, Pipeline, Pipeline_seed
from igf_data.igfdb.igfTables import Project_attribute, Sample_attribute, Experiment_attribute, Seqrun_attribute,Run_attribute
from igf_data.igfdb.baseadaptor import BaseAdaptor
from igf_data.igfdb.projectadaptor import ProjectAdaptor
from igf_data.igfdb.useradaptor import UserAdaptor
from igf_data.igfdb.sampleadaptor import SampleAdaptor
from igf_data.igfdb.platformadaptor import PlatformAdaptor
from igf_data.igfdb.seqrunadaptor import SeqrunAdaptor
from igf_data.igfdb.experimentadaptor import ExperimentAdaptor
from igf_data.igfdb.runadaptor import RunAdaptor
from igf_data.igfdb.collectionadaptor import CollectionAdaptor
from igf_data.igfdb.fileadaptor import FileAdaptor
from igf_data.igfdb.pipelineadaptor import PipelineAdaptor
from igf_data.igfdb.analysisadaptor import AnalysisAdaptor
from sqlalchemy import create_engine

## Define database connection

### MySQL config

In [5]:
dbparams={'dbhost':'0.0.0.0', \
          'dbuser':'igf', \
          'dbpass':'xxxx', \
          'dbname':'igfdb', \
          'driver':'mysql', \
          'connector':'pymysql'}
         # 'engine_config':{'echo':True}} 

### SQLite config

In [55]:
#dbparams={'dbname':'test.db'} #

## Create base adaptor instance

In [6]:
base=BaseAdaptor(**dbparams)
session_class=base.get_session_class()

## Delete existing database and create new

In [7]:
Base.metadata.drop_all(base.engine)
Base.metadata.create_all(base.engine)

## Sequencing platform details for facility

In [8]:
platform_data=[{'platform_igf_id':'ILM4K_001', \
                'model_name':'HISEQ4000',\
                'vendor_name':'ILLUMINA', \
                'software_name':'RTA', \
                'software_version':'RTA2'},
               {'platform_igf_id':'ILMNS_001', \
                'model_name':'NEXTSEQ',\
                'vendor_name':'ILLUMINA', \
                'software_name':'RTA', \
                'software_version':'RTA2'},
               {'platform_igf_id':'ILMMS_001', \
                'model_name':'MISEQ',\
                'vendor_name':'ILLUMINA', \
                'software_name':'RTA', \
                'software_version':'RTA1.18.64'},
              ]

In [9]:
pl=PlatformAdaptor(**{'session_class':base.session_class})
pl.start_session()
pl.store_platform_data(data=platform_data)

### Check sequencing platforms

In [10]:
pl_data=pl.fetch_records(query=pl.session.query(Platform))

In [11]:
pl_data.set_index('platform_id')

,platform_igf_id,model_name,vendor_name,software_name,software_version,date_created
platform_id,,,,,,
1,ILM4K_001,HISEQ4000,ILLUMINA,RTA,RTA2,2017-08-16 15:08:41
2,ILMNS_001,NEXTSEQ,ILLUMINA,RTA,RTA2,2017-08-16 15:08:41
3,ILMMS_001,MISEQ,ILLUMINA,RTA,RTA1.18.64,2017-08-16 15:08:41


In [12]:
pl.close_session()

## Get Project, User and Sample Information

Read all the information from project submission form (modification required)

## Define and store project data

In [13]:
data=[{'project_igf_id':'IGFP0003', \
       'project_name':'project_C',  \
       'description':'Its project C', \
       'project_deadline':'Before August 2017', \
       'comments':'Some samples are treated with drug X'},
      {'project_igf_id':'IGFP0004', \
       'project_name':'project_D',  \
       'description':'Its project D', \
       'project_deadline':'Second week of July', \
       'special_requirement':'Signal files for RNA samples'}]

In [14]:
pa=ProjectAdaptor(**{'session_class':base.session_class})
pa.start_session()
pa.store_project_and_attribute_data(data)

### Check projects

In [15]:
p_data=pa.fetch_records(query=pa.session.query(Project))

In [16]:
p_data.set_index('project_id')

,project_igf_id,project_name,start_timestamp,description,deliverable
project_id,,,,,
1,IGFP0003,project_C,2017-08-16 15:10:01,Its project C,FASTQ
2,IGFP0004,project_D,2017-08-16 15:10:01,Its project D,FASTQ


In [17]:
pa_data=pa.fetch_records(query=pa.session.query(Project_attribute))

In [18]:
pa_data.set_index('project_attribute_id')

,attribute_name,attribute_value,project_id
project_attribute_id,,,
2,comments,Some samples are treated with drug X,1
1,project_deadline,Before August 2017,1
3,project_deadline,Second week of July,2
4,special_requirement,Signal files for RNA samples,2


## Define and store User data

In [19]:
data=[{'user_igf_id':'IGFC0001', \
       'name':'User A', \
       'email_id':'usera@ic.ac.uk', \
       'username':'usera', \
       'password':'passa'}, 
      {'user_igf_id':'IGFC0002', \
       'name':'User B', \
       'email_id':'userb@ic.ac.uk', \
       'username':'userb', \
       'hpc_username':'buser', \
       'password':'passb'}
     ]

### Check users

In [20]:
ua=UserAdaptor(**{'session_class':base.session_class})
ua.start_session()
ua.store_user_data(data=data)

In [21]:
u_data=ua.fetch_records(query=ua.session.query(User))

In [22]:
u_data.set_index('user_id')

,user_igf_id,name,email_id,username,hpc_username,category,status,date_created,password,encryption_salt
user_id,,,,,,,,,,
1,IGFC0001,User A,usera@ic.ac.uk,usera,None,NON_HPC_USER,ACTIVE,2017-08-16 15:10:12,d0c1d497a062b181efd9afcb7ae2f50eeadcb4b7ea3268...,4a29f3c8ca01ad14fe23bbcee1e93b08fe61ba04f4e21a...
2,IGFC0002,User B,userb@ic.ac.uk,userb,buser,HPC_USER,ACTIVE,2017-08-16 15:10:12,e268e5fb093fffaf083eaeefad77eff733a0679baf30f9...,55114b16d46af67fc1a7309a0b65fb91f1c6958616fda8...


In [23]:
ua.close_session()

## Link users to relevant projects

In [24]:
project_user_data=[{'project_igf_id': 'IGFP0003', 'user_igf_id': 'IGFC0001', 'data_authority':True},
                   {'project_igf_id': 'IGFP0003', 'user_igf_id': 'IGFC0002' },
                   {'project_igf_id': 'IGFP0004', 'user_igf_id': 'IGFC0002', 'data_authority':True}
                  ]

In [25]:
pa.assign_user_to_project(data=project_user_data)

In [26]:
pu_data=pa.get_project_user_info()

### Check project users

In [27]:
pu_data.set_index('project_id')

,project_igf_id,project_name,start_timestamp,description,deliverable,user_id,user_igf_id,name,email_id,username,hpc_username,category,status,date_created,password,encryption_salt,data_authority
project_id,,,,,,,,,,,,,,,,,
1,IGFP0003,project_C,2017-08-16 15:10:01,Its project C,FASTQ,1,IGFC0001,User A,usera@ic.ac.uk,usera,None,NON_HPC_USER,ACTIVE,2017-08-16 15:10:12,d0c1d497a062b181efd9afcb7ae2f50eeadcb4b7ea3268...,4a29f3c8ca01ad14fe23bbcee1e93b08fe61ba04f4e21a...,T
1,IGFP0003,project_C,2017-08-16 15:10:01,Its project C,FASTQ,2,IGFC0002,User B,userb@ic.ac.uk,userb,buser,HPC_USER,ACTIVE,2017-08-16 15:10:12,e268e5fb093fffaf083eaeefad77eff733a0679baf30f9...,55114b16d46af67fc1a7309a0b65fb91f1c6958616fda8...,None
2,IGFP0004,project_D,2017-08-16 15:10:01,Its project D,FASTQ,2,IGFC0002,User B,userb@ic.ac.uk,userb,buser,HPC_USER,ACTIVE,2017-08-16 15:10:12,e268e5fb093fffaf083eaeefad77eff733a0679baf30f9...,55114b16d46af67fc1a7309a0b65fb91f1c6958616fda8...,T


In [28]:
pa.close_session()

## Define and store samples

In [29]:
sample_data=[{'sample_igf_id':'IGFS0001', \
              'taxon_id':'9606',\
              'scientific_name':'Homo sapiens',\
              'common_name':'human',\
              'donor_anonymized_id':'donor_001',\
              'description':'Sample A from donor 001',\
              'phenotype':'Healthy',\
              'sex':'FEMALE',\
              'project_igf_id':'IGFP0003',\
              'sample_tube':'tube001',\
              'sample_library':'IGFS0001_20170628'},
             {'sample_igf_id':'IGFS0002', \
              'taxon_id':'9606',\
              'scientific_name':'Homo sapiens',\
              'common_name':'human',\
              'donor_anonymized_id':'donor_002',\
              'description':'Sample B from donor 002',\
              'phenotype':'Cancer',\
              'sex':'FEMALE',\
              'project_igf_id':'IGFP0003',\
              'sample_tube':'tube002',\
              'sample_library':'IGFS0002_20170628'},
            ]

In [30]:
sa=SampleAdaptor(**{'session_class':base.session_class})
sa.start_session()
sa.store_sample_and_attribute_data(data=sample_data)

### Check Samples

In [31]:
s_data=sa.fetch_records(sa.session.query(Sample))

In [32]:
s_data.set_index('sample_id')

,sample_igf_id,taxon_id,scientific_name,common_name,donor_anonymized_id,description,phenotype,sex,status,biomaterial_type,cell_type,tissue_type,cell_line,date_created,project_id
sample_id,,,,,,,,,,,,,,,
1,IGFS0001,9606,Homo sapiens,human,donor_001,Sample A from donor 001,Healthy,FEMALE,ACTIVE,UNKNOWN,None,None,None,2017-08-16 15:10:27,1
2,IGFS0002,9606,Homo sapiens,human,donor_002,Sample B from donor 002,Cancer,FEMALE,ACTIVE,UNKNOWN,None,None,None,2017-08-16 15:10:27,1


In [33]:
sa_data=sa.fetch_records(sa.session.query(Sample_attribute))

In [34]:
sa_data.set_index('sample_attribute_id')

,attribute_name,attribute_value,sample_id
sample_attribute_id,,,
1,sample_library,IGFS0001_20170628,1
2,sample_tube,tube001,1
3,sample_library,IGFS0002_20170628,2
4,sample_tube,tube002,2


In [35]:
sa.close_session()

## Find new sequencing runs and load information

* Look for new sequencing runs (seqrun) in the target directory
* Steps after a new run directory is created
   1. Read the Samplesheet.csv file and get the Project and Samples details
   2. Define new Experiments and Runs based on samplesheet data
    
      Library:    Its an unique library information. I will consider library for each sequencing run as unique
                  e.g. library prepared for sample_A on 07/07/2017 is sample_A_20170707
      Experiment: Its is an unique combination of Sample + Library + Sequencing platform type
                   e.g. a. sample_A has a library sample_A_20170707 and it was sequenced on Hiseq 4000 twice. All the data for sample_A from two sequencing runs will get combined under a single experiment
                        b. Library sample_A_20170707 was sequenced twice on two different platforms,                   it will have two separate experiments for each sequencing runs
       Run:       Fastq files for a sample in a particular lane is considered as a single run
                  e.g. sample_A was present only on lane_1 of a hiseq data, it will have only one run.                  For a NextSeq sequencing run, it will have 4 runs
   3. Generate the fastq files and assign them to relevent 'Run's after the demultiplexing event                  

## Load sequencing Run details

In [36]:
seqrun_data=[{'seqrun_igf_id':'170627_K00345_0012_AHJJKTBBXX', 
              'flowcell_id':'AHJJKTBBXX', 
              'platform_igf_id':'ILM4K_001',
              'Read1':'151', 
              'IndexRead1':'8',
              'IndexRead2':'8',
              'Read2':'151'},
             {'seqrun_igf_id':'170627_M03291_0071_000000000-D0V30', 
              'flowcell_id':'D0V30', 
              'platform_igf_id':'ILM4K_001',
              'Read1':'151', 
              'IndexRead1':'6',
              'IndexRead2':'0',
              'Read2':'0'},
            ]


In [37]:
sra=SeqrunAdaptor(**{'session_class':base.session_class})
sra.start_session()
sra.store_seqrun_and_attribute_data(data=seqrun_data)

### check sequencing runs

In [38]:
sra_data=sra.fetch_records(sra.session.query(Seqrun))

In [39]:
sra_data.set_index('seqrun_id')

,seqrun_igf_id,reject_run,date_created,flowcell_id,platform_id
seqrun_id,,,,,
1,170627_K00345_0012_AHJJKTBBXX,N,2017-08-16 15:12:23,AHJJKTBBXX,1
2,170627_M03291_0071_000000000-D0V30,N,2017-08-16 15:12:23,D0V30,1


In [47]:
sra_at_data=sra.fetch_records(sra.session.query(Seqrun_attribute))

In [41]:
sra_at_data.set_index('seqrun_attribute_id')

,attribute_name,attribute_value,seqrun_id
seqrun_attribute_id,,,
3,IndexRead1,8,1
2,IndexRead2,8,1
4,Read1,151,1
1,Read2,151,1
7,IndexRead1,6,2
6,IndexRead2,0,2
8,Read1,151,2
5,Read2,0,2


In [42]:
seqrun_stats_data=[{'seqrun_igf_id':'170627_K00345_0012_AHJJKTBBXX',
                    'lane_number':'1',
                    'bases_mask':'y150n1,i8,i8,y150n1',
                    'undetermined_barcodes':[{'barcode':'ATTTA+GGGC','count':50,},
                                             {'barcode':'TTTA+CCCG','count':150}],
                    'known_barcodes':[{'barcode':'AAAAA+TTTT','count':10,},
                                      {'barcode':'CCCC+GGGG','count':20}],
                    'undetermined_fastqc':{'read1':{'number':130},
                                           'read2':{'number':130}},
                   }
                  ]

In [43]:
sra.store_seqrun_stats_data(data=seqrun_stats_data)

In [48]:
sra_st_data=sra.fetch_records(sra.session.query(Seqrun_stats))

In [49]:
sra_st_data.set_index('seqrun_stats_id')

,seqrun_id,lane_number,bases_mask,undetermined_barcodes,known_barcodes,undetermined_fastqc
seqrun_stats_id,,,,,,
1,1,1,"y150n1,i8,i8,y150n1","[{'count': 50, 'barcode': 'ATTTA+GGGC'}, {'cou...","[{'count': 10, 'barcode': 'AAAAA+TTTT'}, {'cou...","{'read1': {'number': 130}, 'read2': {'number':..."


In [50]:
sra.close_session()

## Define and store experiment

In [51]:
exp_data=[{'experiment_igf_id':'IGFS0001_20170628_IH4',
           'library_name':'IGFS0001_20170628',
           'library_strategy':'RNA-SEQ',
           'experiment_type':'TOTAL-RNA',
           'library_layout':'PAIRED',
           'project_igf_id':'IGFP0003',
           'sample_igf_id':'IGFS0001',
           'platform_name':'HISEQ4000'},
          {'experiment_igf_id':'IGFS0002_20170628_IH4',
           'library_name':'IGFS0002_20170628',
           'library_strategy':'RNA-SEQ',
           'experiment_type':'TOTAL-RNA',
           'library_layout':'PAIRED',
           'project_igf_id':'IGFP0003',
           'sample_igf_id':'IGFS0002',
           'platform_name':'HISEQ4000'},
          ]

In [52]:
ea=ExperimentAdaptor(**{'session_class':base.session_class})
ea.start_session()
ea.store_project_and_attribute_data(data=exp_data)

### Check experiments

In [53]:
e_data=ea.fetch_records(ea.session.query(Experiment))

In [54]:
e_data.set_index('experiment_id')

,experiment_igf_id,project_id,sample_id,library_name,library_source,library_strategy,experiment_type,library_layout,status,date_created,platform_name
experiment_id,,,,,,,,,,,
1,IGFS0001_20170628_IH4,1,1,IGFS0001_20170628,UNKNOWN,RNA-SEQ,TOTAL-RNA,PAIRED,ACTIVE,2017-08-16 15:15:46,HISEQ4000
2,IGFS0002_20170628_IH4,1,2,IGFS0002_20170628,UNKNOWN,RNA-SEQ,TOTAL-RNA,PAIRED,ACTIVE,2017-08-16 15:15:46,HISEQ4000


In [55]:
ea_data=ea.fetch_records(query=ea.session.query(Experiment_attribute))

In [56]:
ea_data

,experiment_attribute_id,attribute_name,attribute_value,experiment_id


In [57]:
ea.close_session()

## Define and store run

In [58]:
run_data=[{'run_igf_id':'IGFS0001_20170628_IH4_L1',
           'experiment_igf_id':'IGFS0001_20170628_IH4',
           'seqrun_igf_id':'170627_M03291_0071_000000000-D0V30',
           'lane_number':'1',
           'run_barcode':'BARCODE_X'},
          {'run_igf_id':'IGFS0002_20170628_IH4_L2',
           'experiment_igf_id':'IGFS0002_20170628_IH4',
           'seqrun_igf_id':'170627_M03291_0071_000000000-D0V30',
           'lane_number':'2',
           'run_barcode':'BARCODE_Y'},
         ]

In [59]:
ra=RunAdaptor(**{'session_class':base.session_class})
ra.start_session()
ra.store_run_and_attribute_data(data=run_data)

### Check runs

In [60]:
r_data=ra.fetch_records(query=ra.session.query(Run))

In [61]:
r_data.set_index('run_id')

,run_igf_id,experiment_id,seqrun_id,status,lane_number,date_created
run_id,,,,,,
1,IGFS0001_20170628_IH4_L1,1,2,ACTIVE,1,2017-08-16 15:15:58
2,IGFS0002_20170628_IH4_L2,2,2,ACTIVE,2,2017-08-16 15:15:58


In [62]:
ra_data=ra.fetch_records(query=ra.session.query(Run_attribute))

In [63]:
ra_data.set_index('run_attribute_id')

,attribute_name,attribute_value,run_id
run_attribute_id,,,
1,run_barcode,BARCODE_X,1
2,run_barcode,BARCODE_Y,2


In [64]:
ra.close_session()

## Check combined data for projects

In [65]:
base.start_session()
all_data=base.fetch_records(base.session.query(Project.project_igf_id, \
                                               Sample.sample_igf_id, \
                                               Experiment.experiment_igf_id, \
                                               Run.run_igf_id).\
                                               join(Sample).\
                                               join(Experiment).\
                                               join(Run))

In [66]:
all_data.set_index('project_igf_id')

,sample_igf_id,experiment_igf_id,run_igf_id
project_igf_id,,,
IGFP0003,IGFS0001,IGFS0001_20170628_IH4,IGFS0001_20170628_IH4_L1
IGFP0003,IGFS0002,IGFS0002_20170628_IH4,IGFS0002_20170628_IH4_L2


In [67]:
base.close_session()

In [68]:
collection_data=[{'name':'IGFS0001','type':'SAMPLE_BAM','table':'sample','aligner_name':'bwa'},
                 {'name':'IGFS0002','type':'SAMPLE_BAM','table':'sample','aligner_name':'star'},
                ]

In [69]:
ca=CollectionAdaptor(**{'session_class':base.session_class})
ca.start_session()
ca.store_collection_and_attribute_data(data=collection_data)

In [70]:
file_data=[{'file_path':'/demo/files1','file_attribute1':'its attribute1'},
           {'file_path':'/demo/files2','file_attribute2':'its attribute2'},]

In [71]:
fa=FileAdaptor(**{'session_class':base.session_class})
fa.start_session()

In [72]:
fa.store_file_and_attribute_data(data=file_data)
fa.close_session()

In [73]:
collection_files_data=[{'name':'IGFS0001','type':'SAMPLE_BAM','file_path':'/demo/files1'},
                       {'name':'IGFS0002','type':'SAMPLE_BAM', 'file_path':'/demo/files2'},
                      ]

In [74]:
ca.create_collection_group(data=collection_files_data)

In [75]:
c_files=ca.get_collection_files(collection_name='IGFS0001', collection_type='SAMPLE_BAM')

//home/vmuser/.pyenv/versions/3.5.2/lib/python3.5/site-packages/sqlalchemy/sql/base.py:524: SAWarning: Column 'type' on table <sqlalchemy.sql.selectable.Select at 0x7fc7a681bbe0; Select object> being replaced by Column('type', Enum('BCL_DIR', 'BCL_TAR', 'FASTQ_TAR', 'FASTQC_TAR', 'FASTQ', 'FASTQGZ', 'BAM', 'CRAM', 'GFF', 'BED', 'GTF', 'FASTA', 'UNKNOWN'), table=<Select object>, nullable=False), which has the same key.  Consider use_labels for select() statements.
  (key, getattr(existing, 'table', None), value))


In [76]:
c_files

,collection_id,name,type,table,date_stamp,file_id,file_path,location,type,md5,size,date_created,date_updated
0,1,IGFS0001,SAMPLE_BAM,sample,2017-08-16 15:17:03,1,/demo/files1,UNKNOWN,UNKNOWN,None,None,2017-08-16 15:17:42,2017-08-16 15:17:42


In [77]:
ca.close_session()

In [78]:
pipeline_data=[{'pipeline_name':'demultiplexing_20170731', 
                'pipeline_db':'sqlite:////path/bcl2fastq.db', 
                'pipeline_init_conf':{'input_dir':'/path/input_dir',
                                      'output_dir':'/path/output_dir'}, 
                'pipeline_run_conf':{'output_dir':'/path/output_dir2'},}]

In [79]:
pipeline_seed_data=[{'pipeline_name':'demultiplexing_20170731', 'seed_id':'1', 'seed_table':'seqrun'},
                    {'pipeline_name':'demultiplexing_20170731', 'seed_id':'2', 'seed_table':'seqrun'},
                   ]

In [80]:
pla=PipelineAdaptor(**{'session_class':base.session_class})
pla.start_session()
pla.store_pipeline_data(data=pipeline_data)

In [82]:
pla_data=pla.fetch_records(query=pla.session.query(Pipeline))

In [83]:
pla_data.set_index('pipeline_id')

,pipeline_name,pipeline_db,pipeline_init_conf,pipeline_run_conf,pipeline_type,is_active,date_stamp
pipeline_id,,,,,,,
1,demultiplexing_20170731,sqlite:////path/bcl2fastq.db,"{'input_dir': '/path/input_dir', 'output_dir':...",{'output_dir': '/path/output_dir2'},EHIVE,Y,2017-08-16 15:21:24


In [84]:
pla.create_pipeline_seed(data=pipeline_seed_data)

In [85]:
pls_data=pla.fetch_records(query=pla.session.query(Pipeline_seed))

In [87]:
pls_data.set_index('pipeline_seed_id')

,seed_id,seed_table,pipeline_id,status,date_stamp
pipeline_seed_id,,,,,
1,1,SEQRUN,1,SEEDED,2017-08-16 15:23:28
2,2,SEQRUN,1,SEEDED,2017-08-16 15:23:28


In [88]:
update_pipeline_seed_data=[{'pipeline_name':'demultiplexing_20170731', 
                            'seed_id':'1', 
                            'seed_table':'seqrun', 
                            'status':'RUNNING'},
                           {'pipeline_name':'demultiplexing_20170731', 
                            'seed_id':'2', 
                            'seed_table':'seqrun', 
                            'status':'FAILED'},
                          ]

In [89]:
pla.update_pipeline_seed(data=update_pipeline_seed_data)

In [90]:
pls_data=pla.fetch_records(query=pla.session.query(Pipeline_seed))

In [91]:
pls_data.set_index('pipeline_seed_id')

,seed_id,seed_table,pipeline_id,status,date_stamp
pipeline_seed_id,,,,,
1,1,SEQRUN,1,RUNNING,2017-08-16 14:24:44
2,2,SEQRUN,1,FAILED,2017-08-16 14:24:44


In [92]:
pla.close_session()

In [93]:
analysis_data=[{'project_igf_id':'IGFP0003', 
                'analysis_type':'RNA_DIFFERENTIAL_EXPRESSION', 
                'analysis_description':[{'CONTROL':'IGFS0001','CASE' :'IGFS0002'},] }]

In [94]:
aa=AnalysisAdaptor(**{'session_class':base.session_class})
aa.start_session()
aa.store_analysis_data(data=analysis_data)

In [95]:
da=aa.fetch_analysis_records_project_igf_id(project_igf_id='IGFP0003')

In [96]:
da

,project_igf_id,analysis_type,analysis_description
0,IGFP0003,RNA_DIFFERENTIAL_EXPRESSION,"[{'CASE': 'IGFS0002', 'CONTROL': 'IGFS0001'}]"


In [97]:
aa.close_session()